In [ ]:
import cuml
from cuml.ensemble import RandomForestClassifier as cuMLRandomForestClassifier
from cuml.metrics import accuracy_score as cuml_accuracy_score, f1_score as cuml_f1_score, recall_score as cuml_recall_score
import numpy as np
import pandas as pd
import cupy as cp
from skimage.feature import hog
import joblib
from collections import Counter
import cv2
from cuml.model_selection import train_test_split

height, width = 10, 10

In [5]:
data = pd.read_csv(f"../archive/ascii_character_classification_{height}_x_{width}.csv", header=0).sample(frac=.05)
label_counts = Counter(data.iloc[:, 0])
print(label_counts)


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 22+36-2370, mixed mode, sharing)
  Starting server from C:\Users\Sai\AppData\Local\Programs\Python\Python312\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Sai\AppData\Local\Temp\tmp584dvni3
  JVM stdout: C:\Users\Sai\AppData\Local\Temp\tmp584dvni3\h2o_Sai_started_from_python.out
  JVM stderr: C:\Users\Sai\AppData\Local\Temp\tmp584dvni3\h2o_Sai_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,America/Chicago
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,1 month and 21 days
H2O_cluster_name:,H2O_from_python_Sai_0tpf52
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.929 Gb
H2O_cluster_total_cores:,0
H2O_cluster_allowed_cores:,0
H2O_cluster_status:,"locked, healthy"


Counter({0: 2587, 26: 270, 8: 270, 84: 267, 71: 266, 57: 265, 32: 264, 12: 264, 51: 263, 88: 262, 69: 261, 77: 261, 6: 260, 61: 258, 83: 257, 9: 256, 40: 254, 11: 254, 92: 253, 27: 251, 16: 250, 10: 247, 64: 247, 66: 246, 4: 246, 31: 246, 90: 245, 76: 245, 91: 245, 39: 245, 60: 244, 29: 244, 30: 243, 37: 243, 87: 243, 82: 242, 17: 242, 5: 242, 62: 241, 14: 241, 74: 240, 95: 240, 89: 240, 55: 240, 63: 237, 18: 237, 85: 237, 21: 236, 53: 235, 7: 235, 34: 234, 81: 234, 48: 233, 80: 233, 20: 233, 54: 232, 93: 232, 38: 232, 56: 232, 49: 231, 22: 231, 52: 231, 15: 231, 35: 231, 2: 230, 58: 230, 19: 230, 42: 229, 24: 229, 23: 228, 75: 228, 78: 227, 50: 227, 65: 226, 44: 223, 33: 222, 59: 221, 28: 220, 68: 220, 47: 219, 72: 219, 3: 219, 36: 216, 25: 216, 1: 214, 86: 212, 45: 211, 73: 209, 67: 208, 70: 208, 46: 208, 41: 207, 43: 206, 79: 204, 13: 199, 94: 178})


In [6]:
# Move to GPU by converting to cupy arrays (cuML uses GPU arrays via CuPy)
X = cp.array(data.iloc[:, 1:].astype("float64"))
y = cp.array(data.iloc[:, 0].astype("float64"))

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
# cuML Random Forest Classifier
clf = cuMLRandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Save the model
joblib.dump(clf, '../artifacts/random_forest_model_10_x_10_cuml.pkl')


In [9]:
# Predict and evaluate
y_pred_train = clf.predict(X_train)
train_accuracy = cuml_accuracy_score(y_train, y_pred_train)

y_pred_test = clf.predict(X_test)
test_accuracy = cuml_accuracy_score(y_test, y_pred_test)

f1 = cuml_f1_score(y_test, y_pred_test, average='weighted')
recall = cuml_recall_score(y_test, y_pred_test, average='weighted')

print(f"Train Accuracy: {train_accuracy*100:.4f}%")
print(f"Test Accuracy: {test_accuracy*100:.4f}%")
print(f"F1 Score: {f1*100:.4f}%")
print(f"Recall: {recall*100:.4f}%")

drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


AttributeError: type object 'MetricsBase' has no attribute 'accuracy'

In [4]:
def extract_hog_features(images):
    hog_features = []
    for image in images:
        image_reshaped = image.reshape((height, width))
        features = hog(image_reshaped, pixels_per_cell=(2, 2), cells_per_block=(1, 1), feature_vector=True)
        hog_features.append(features)
    return np.array(hog_features)

# Move HOG features extraction to GPU (optional if needed)
X_hog = extract_hog_features(cp.asnumpy(X))  # Convert to CPU to use scikit-image
X_hog = cp.array(X_hog)  # Convert back to GPU

X_train, X_test, y_train, y_test = train_test_split(X_hog, y, test_size=0.2, random_state=42)


NameError: name 'clf' is not defined

In [ ]:
# cuML Random Forest Classifier for HOG features
clf_hog = cuMLRandomForestClassifier(n_estimators=100, random_state=42)
clf_hog.fit(X_train, y_train)

# Save the HOG model
joblib.dump(clf_hog, '../artifacts/random_forest_model_hog_10_x_10_cuml.pkl')


In [ ]:
y_pred_train_hog = clf_hog.predict(X_train)
train_accuracy_hog = cuml_accuracy_score(y_train, y_pred_train_hog)

y_pred_test_hog = clf_hog.predict(X_test)
test_accuracy_hog = cuml_accuracy_score(y_test, y_pred_test_hog)

f1_hog = cuml_f1_score(y_test, y_pred_test_hog, average='weighted')
recall_hog = cuml_recall_score(y_test, y_pred_test_hog, average='weighted')

print(f"Train Accuracy (HOG): {train_accuracy_hog*100:.4f}%")
print(f"Test Accuracy (HOG): {test_accuracy_hog*100:.4f}%")
print(f"F1 Score (HOG): {f1_hog*100:.4f}%")
print(f"Recall (HOG): {recall_hog*100:.4f}%")


In [ ]:
# Extract SIFT features
def extract_sift_features(images):
    sift = cv2.SIFT_create()
    sift_features = []
    
    for image in images:
        image_reshaped = image.reshape((height, width)).astype(np.uint8)
        keypoints, descriptors = sift.detectAndCompute(image_reshaped, None)
        
        if descriptors is None:
            descriptors = np.zeros((1, sift.descriptorSize()), dtype=np.float32)
        
        features = descriptors.flatten()
        sift_features.append(features)
    
    return np.array(sift_features)

X_sift = extract_sift_features(cp.asnumpy(X))  # Convert to CPU for OpenCV
X_sift = cp.array(X_sift)  # Convert back to GPU

# Pad descriptors to consistent length
max_len = max(len(f) for f in X_sift)
X_sift = cp.array([np.pad(f, (0, max_len - len(f)), 'constant') for f in X_sift])

X_train, X_test, y_train, y_test = train_test_split(X_sift, y, test_size=0.2, random_state=42)


In [ ]:
# cuML Random Forest Classifier for SIFT features
clf_sift = cuMLRandomForestClassifier(n_estimators=100, random_state=42)
clf_sift.fit(X_train, y_train)

# Save the SIFT model
joblib.dump(clf_sift, '../artifacts/random_forest_model_sift_10_x_10_cuml.pkl')

# %%
y_pred_train_sift = clf_sift.predict(X_train)
train_accuracy_sift = cuml_accuracy_score(y_train, y_pred_train_sift)

y_pred_test_sift = clf_sift.predict(X_test)
test_accuracy_sift = cuml_accuracy_score(y_test, y_pred_test_sift)

f1_sift = cuml_f1_score(y_test, y_pred_test_sift, average='weighted')
recall_sift = cuml_recall_score(y_test, y_pred_test_sift, average='weighted')

print(f"Train Accuracy (SIFT): {train_accuracy_sift*100:.4f}%")
print(f"Test Accuracy (SIFT): {test_accuracy_sift*100:.4f}%")
print(f"F1 Score (SIFT): {f1_sift*100:.4f}%")
print(f"Recall (SIFT): {recall_sift*100:.4f}%")